In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ner-dataset/ner_datasetreference.csv


In [ ]:
!pip install simpletransformers

In [3]:
from simpletransformers.ner import NERModel,NERArgs
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Reading Data and Preprocessing

In [6]:
df = pd.read_csv("/kaggle/input/ner-dataset/ner_datasetreference.csv",encoding='latin1')
df.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 4 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   Sentence #  47959 non-null    object
 1   Word        1048565 non-null  object
 2   POS         1048575 non-null  object
 3   Tag         1048575 non-null  object
dtypes: object(4)
memory usage: 32.0+ MB


In [8]:
df.isnull().sum()

Sentence #    1000616
Word               10
POS                 0
Tag                 0
dtype: int64

In [9]:
df['Sentence #'].fillna(method = "ffill",inplace = True)

/tmp/ipykernel_42/3710866575.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Sentence #'].fillna(method = "ffill",inplace = True)


In [10]:
df.isnull().sum()

Sentence #     0
Word          10
POS            0
Tag            0
dtype: int64

In [11]:
df.dropna(inplace = True)

In [12]:
df.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O


## Convert Text to Numerical Representation

In [13]:
df["Sentence #"] = LabelEncoder().fit_transform(df["Sentence #"])

In [14]:
df.head()

,Sentence #,Word,POS,Tag
0,0,Thousands,NNS,O
1,0,of,IN,O
2,0,demonstrators,NNS,O
3,0,have,VBP,O
4,0,marched,VBN,O


## Separate Features and labels and Spilt Train and Test Datasets

In [15]:
X= df[["Sentence #","Word"]]
Y =df["Tag"]

In [16]:
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size =0.2)

## Create Datasets for Transformer accepted format 

In [26]:
train_data = pd.DataFrame({"sentence_id":x_train["Sentence #"],"words":x_train["Word"],"labels":y_train})
test_data = pd.DataFrame({"sentence_id":x_test["Sentence #"],"words":x_test["Word"],"labels":y_test})

In [27]:
train_data.head()

,sentence_id,words,labels
75814,27090,of,O
579946,18351,.,O
1001940,39798,Malawi,B-geo
470753,12809,He,O
341138,6232,be,O


In [28]:
test_data.head()

,sentence_id,words,labels
632447,21015,$,O
618425,20308,Nigerian,B-gpe
375499,7976,'s,O
392932,8850,'s,O
75588,26979,of,O


In [29]:
# unique Labels 17
labels = y_train.unique().tolist()
labels,len(labels)

(['O',
  'B-geo',
  'I-org',
  'B-gpe',
  'B-per',
  'B-tim',
  'I-tim',
  'B-org',
  'I-per',
  'I-geo',
  'I-eve',
  'I-gpe',
  'B-eve',
  'I-art',
  'B-art',
  'B-nat',
  'I-nat'],
 17)

## Model Arguments Updating for training

In [30]:
args = NERArgs()
args.num_train_epochs = 1
args.learning_rate = 1e-4
args.overwrite_output_dir =True
args.train_batch_size = 32
args.eval_batch_size = 32

In [31]:
model = NERModel('bert', 'bert-base-cased',labels=labels,args =args)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Training Process started

In [32]:
model.train_model(train_data,eval_data = test_data,acc=accuracy_score)

  0%|          | 0/4 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/1499 [00:00<?, ?it/s]

(1499, 0.19214358137022106)

## Model Evaluation

In [33]:
result, model_outputs, preds_list = model.eval_model(test_data)

  0%|          | 0/5 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1460 [00:00<?, ?it/s]

In [36]:
result

{'eval_loss': 0.17229109975343493,
 'precision': 0.8258264534782457,
 'recall': 0.7572911675451206,
 'f1_score': 0.7900753182696795}

## Model Predictions

In [34]:
prediction, model_output = model.predict(["""Elon Reeve Musk is a businessman and investor. He is the founder, chairman, CEO, and chief technology officer of SpaceX; angel investor, CEO, product architect and former chairman of Tesla, Inc.; owne"""])

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [35]:
prediction

[[{'Elon': 'B-per'},
  {'Reeve': 'B-per'},
  {'Musk': 'I-per'},
  {'is': 'O'},
  {'a': 'O'},
  {'businessman': 'O'},
  {'and': 'O'},
  {'investor.': 'O'},
  {'He': 'O'},
  {'is': 'O'},
  {'the': 'O'},
  {'founder,': 'O'},
  {'chairman,': 'O'},
  {'CEO,': 'O'},
  {'and': 'O'},
  {'chief': 'O'},
  {'technology': 'O'},
  {'officer': 'O'},
  {'of': 'O'},
  {'SpaceX;': 'B-org'},
  {'angel': 'O'},
  {'investor,': 'O'},
  {'CEO,': 'O'},
  {'product': 'O'},
  {'architect': 'O'},
  {'and': 'O'},
  {'former': 'O'},
  {'chairman': 'O'},
  {'of': 'O'},
  {'Tesla,': 'B-org'},
  {'Inc.;': 'I-org'},
  {'owne': 'O'}]]